In [ ]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [1]:
!pip install opendatasets
!pip install pandas

Скачиваем датасеты

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: angrytea
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025


100%|██████████| 9.57M/9.57M [00:00<00:00, 1.93GB/s]

Библиотеки

In [3]:
import opendatasets as od
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.base import BaseEstimator, ClassifierMixin

Открываем датасет  
Удаляем строки, не имеющие в Members значения  
Задаем условие успешности: тайтл успешный, если у него 75% и больше зрителей: 1 - аниме успешное (собрало много просмотров) 0 - не успешное  
Для признаков возьмем Source, Type, Released_Season, Episodes, Genres, Studios, Demographic  
Пропуски по эпизодам заменяем медианой, по категориальным признакам на Unknown, жанрам и студиям поставим пустые строки  
Создаем список жанров  
Ищем самые популярные жанры и делаем для них колонки  
Список студий делаем  
Ищем популярные студии, не учитывая 'add some'  
Задаем колонки  
Собираем матрицу X из фич и назначаем целевую переменную успешности y  
Делим данные на train и test  
Числовые и бинарные признаки будем масштабировать, а для категориальных используем one-hot encoding  

In [4]:
pd.set_option("display.max_columns", None)
df = pd.read_csv("myanimelist-2025/mal_anime.csv")
df["Members"] = pd.to_numeric(df["Members"], errors="coerce")
df = df.dropna(subset=["Members"])
th = df["Members"].quantile(0.75)

df["hit"] = (df["Members"] >= th).astype(int)
df["Episodes"] = pd.to_numeric(df["Episodes"], errors="coerce")
m_episodes = df["Episodes"].median()
df["Episodes"] = df["Episodes"].fillna(m_episodes)

for col in ["Source", "Type", "Released_Season", "Demographic"]:
    df[col] = df[col].fillna("Unknown")

for col in ["Genres", "Studios"]: df[col] = df[col].fillna("")

df["Genres_l"] = df["Genres"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])
genre_count = Counter()

for g in df["Genres_l"]:
    genre_count.update(g)

top_g = [i for i, _ in genre_count.most_common(10)]

for g in top_g:
    col_name = f"genre_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))

g_cols = [f"genre_{g}" for g in top_g]
df["Studios_l"] = df["Studios"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])

studio_count = Counter()
for s in df["Studios_l"]:
  for st in s:
    clean_s = st.strip()
    if "add some" in clean_s.lower(): continue
    studio_count[clean_s] += 1

top_s = [i for i, _ in studio_count.most_common(10)]
for s in top_s:
    col_name = f"studio_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))

studio_cols = [f"studio_{s}" for s in top_s]

num = ["Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature = num + categ

X = df[feature]
y = df["hit"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
pred = ColumnTransformer(transformers=[
    ("num", StandardScaler(),num),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ)
])

Строим базовый RandomForestClassifier с типичными настройками (100 деревьев, критерий gini, без учета дисбаланса классов), обучаю его на тренировочной выборке

Cлучайный лес даёт довольно высокую общую точность (0.74) и неплохой ROC-AUC (0.71), но F1 по классу «hit» остаётся низким (0.35). Модель хорошо распознаёт неуспешные тайтлы (класс 0: precision ≈ 0.79, recall ≈ 0.89) и сильно перекошена в их сторону, при этом часто «промахивается» по успешным аниме (класс 1: recall всего 0.28). То есть базовая конфигурация леса по сути ведёт себя как ещё один консервативный фильтр, который мало рискует предсказывать «hit»

In [ ]:
rf_base = Pipeline(steps=[
    ("pred", pred),
    ("clf", RandomForestClassifier(
        n_estimators=100,
        criterion="gini",
        max_depth=None,
        random_state=42,
        n_jobs=-1
    ))
])

rf_base.fit(X_train, y_train)

y_pred_base = rf_base.predict(X_test)
y_proba_base = rf_base.predict_proba(X_test)[:, 1]

print("Бейзлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_base))
print("F1-score:", f1_score(y_test, y_pred_base))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_base))
print(classification_report(y_test, y_pred_base))


Бейзлайн
Accuracy: 0.7385892116182573
F1-score: 0.3531827515400411
ROC-AUC: 0.7065466106356295
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       903
           1       0.46      0.28      0.35       302

    accuracy                           0.74      1205
   macro avg       0.63      0.59      0.59      1205
weighted avg       0.71      0.74      0.72      1205



Первая гипотеза  
В сетку включаю число деревьев, максимальную глубину, ограничение на размер узлов и стратегию выбора признаков, а также class_weight (None/balanced). Подбираю параметры через GridSearchCV по F1-метрике

F1 по классу hit заметно растёт (с 0.35 до 0.49) и ROC-AUC тоже улучшается (0.73 против 0.71), но за это мы платим падением общей точности до 0.67. То есть модель начинает активнее ловить редкий класс (recall для hit вырастает до 0.63), но начинает чаще ошибаться по неуспешным тайтлам. Такой режим более полезен, если важнее не пропускать потенциальные «хиты», чем поддерживать максимальную accuracy

In [ ]:
param_grid_rf_1 = {
    "clf__n_estimators": [100, 300],
    "clf__max_depth": [None, 10, 20],
    "clf__min_samples_split": [2, 10, 20],
    "clf__min_samples_leaf": [1, 2, 5],
    "clf__max_features": ["sqrt", "log2"],
    "clf__class_weight": [None, "balanced"],
}

rf_clf_1 = Pipeline(steps=[
    ("pred", pred),
    ("clf", RandomForestClassifier(
        random_state=42,
        n_jobs=-1
    ))
])

grid_rf_1 = GridSearchCV(
    estimator=rf_clf_1,
    param_grid=param_grid_rf_1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_rf_1.fit(X_train, y_train)

print("Лучшие параметры:", grid_rf_1.best_params_)
print("Лучший F1 на CV:", grid_rf_1.best_score_)

best_rf_1 = grid_rf_1.best_estimator_
y_pred_rf_1 = best_rf_1.predict(X_test)
y_proba_rf_1 = best_rf_1.predict_proba(X_test)[:, 1]

print("\nГипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_rf_1))
print("F1-score:", f1_score(y_test, y_pred_rf_1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_rf_1))
print(classification_report(y_test, y_pred_rf_1))


Лучшие параметры: {'clf__class_weight': 'balanced', 'clf__max_depth': 10, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 20, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.488091410462309

Гипотеза 1
Accuracy: 0.674688796680498
F1-score: 0.4909090909090909
ROC-AUC: 0.7296869155794151
              precision    recall  f1-score   support

           0       0.85      0.69      0.76       903
           1       0.40      0.63      0.49       302

    accuracy                           0.67      1205
   macro avg       0.63      0.66      0.63      1205
weighted avg       0.74      0.67      0.69      1205



Вторая гипотеза

Небольшой, но устойчивый прирост качества: F1 поднимается до 0.51, ROC-AUC — до 0.74, а accuracy остаётся на уровне 0.69. Распределение по классам становится более сбалансированным: модель немного улучшает и способность находить успешные тайтлы (recall 0.64), и общее качество. Логарифмирование длины сериала помогает лесу лучше учитывать различия между короткими и длинными тайтлами, не давая одному признаку доминировать

In [ ]:
df2 = df.copy()
df2["log_Episodes"] = np.log1p(df2["Episodes"])

num2 = ["log_Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature2 = num2 + categ

X2 = df2[feature2]
y2 = df2["hit"]

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2,
    test_size=0.2,
    random_state=42,
    stratify=y2
)

pred2 = ColumnTransformer(
    transformers=[
        ("num2", StandardScaler(), num2),
        ("categ", OneHotEncoder(handle_unknown="ignore"), categ),
    ]
)

rf_clf_2 = Pipeline(steps=[
    ("pred", pred2),
    ("clf", RandomForestClassifier(
        random_state=42,
        n_jobs=-1
    ))
])

grid_rf_2 = GridSearchCV(
    estimator=rf_clf_2,
    param_grid=param_grid_rf_1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_rf_2.fit(X2_train, y2_train)

print("Лучшие параметры:", grid_rf_2.best_params_)
print("Лучший F1 на CV:", grid_rf_2.best_score_)

best_rf_2 = grid_rf_2.best_estimator_
y_pred_rf_2 = best_rf_2.predict(X2_test)
y_proba_rf_2 = best_rf_2.predict_proba(X2_test)[:, 1]

print("\nRandomForest + log(Episodes)")
print("Accuracy:", accuracy_score(y2_test, y_pred_rf_2))
print("F1-score:", f1_score(y2_test, y_pred_rf_2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_rf_2))
print(classification_report(y2_test, y_pred_rf_2))


Лучшие параметры: {'clf__class_weight': 'balanced', 'clf__max_depth': 20, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 5, 'clf__min_samples_split': 20, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.4885597072008033

RandomForest + log(Episodes)
Accuracy: 0.6879668049792531
F1-score: 0.5078534031413613
ROC-AUC: 0.736859475039053
              precision    recall  f1-score   support

           0       0.85      0.70      0.77       903
           1       0.42      0.64      0.51       302

    accuracy                           0.69      1205
   macro avg       0.64      0.67      0.64      1205
weighted avg       0.75      0.69      0.71      1205



Третья гипотеза

Качество в целом остаётся на уровне гипотезы 1: F1 ≈ 0.485, ROC-AUC ≈ 0.73, accuracy ≈ 0.67. Модель по-прежнему лучше бейзлайна по F1, но чуть уступает конфигурации с логарифмом эпизодов. То есть агрессивное упрощение многомерных жанровых/студийных признаков до top-20 само по себе не даёт дополнительного выигрыша по сравнению с уже настроенным лесом; ключевой вклад в улучшение качества здесь делает балансировка классов и тюнинг параметров, а не срезание «хвоста» по жанрам и студиям

In [ ]:
top_g20 = [g for g, _ in genre_count.most_common(20)]
for g in top_g20:
    col_name = f"genre_{g}"
    if col_name not in df.columns:
        df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))
g_cols20 = [f"genre_{g}" for g in top_g20]

top_s20 = [s for s, _ in studio_count.most_common(20)]
for s in top_s20:
    col_name = f"studio_{s}"
    if col_name not in df.columns:
        df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))
studio_cols20 = [f"studio_{s}" for s in top_s20]

num3 = ["Episodes"] + g_cols20 + studio_cols20
categ3 = ["Source", "Type", "Released_Season", "Demographic"]
feature3 = num3 + categ3

X3 = df[feature3]
y3 = df["hit"]

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3,
    test_size=0.2,
    random_state=42,
    stratify=y3
)

pred3 = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num3),
        ("categ", OneHotEncoder(handle_unknown="ignore"), categ3),
    ]
)

rf_clf_3 = Pipeline(steps=[
    ("pred", pred3),
    ("clf", RandomForestClassifier(
        random_state=42,
        n_jobs=-1
    ))
])

grid_rf_3 = GridSearchCV(
    estimator=rf_clf_3,
    param_grid=param_grid_rf_1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_rf_3.fit(X3_train, y3_train)

print("Лучшие параметры:", grid_rf_3.best_params_)
print("Лучший F1 на CV:", grid_rf_3.best_score_)

best_rf_3 = grid_rf_3.best_estimator_
y_pred_rf_3 = best_rf_3.predict(X3_test)
y_proba_rf_3 = best_rf_3.predict_proba(X3_test)[:, 1]

print("\nRandomForest + top-20 жанров/студий")
print("Accuracy:", accuracy_score(y3_test, y_pred_rf_3))
print("F1-score:", f1_score(y3_test, y_pred_rf_3))
print("ROC-AUC:", roc_auc_score(y3_test, y_proba_rf_3))
print(classification_report(y3_test, y_pred_rf_3))


Лучшие параметры: {'clf__class_weight': 'balanced', 'clf__max_depth': 10, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 20, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.49006025697151706

RandomForest + top-20 жанров/студий
Accuracy: 0.6721991701244814
F1-score: 0.485006518904824
ROC-AUC: 0.7277085946037125
              precision    recall  f1-score   support

           0       0.84      0.69      0.76       903
           1       0.40      0.62      0.49       302

    accuracy                           0.67      1205
   macro avg       0.62      0.65      0.62      1205
weighted avg       0.73      0.67      0.69      1205



Собственная реализация

In [5]:
class MyDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        criterion="gini",
        random_state=None
    ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        self.n_classes_ = self.classes_.shape[0]
        if self.n_classes_ != 2:
            raise ValueError("MyDecisionTreeClassifier сейчас поддерживает только бинарную классификацию")

        if self.criterion != "gini":
            raise ValueError("В данной реализации поддерживается только criterion='gini'")

        rng = np.random.RandomState(self.random_state)

        self.n_features_ = X.shape[1]
        self.tree_ = self._build_tree(X, y, depth=0, rng=rng)
        return self

    def _gini(self, y):
        m = y.shape[0]
        if m == 0:
            return 0.0
        _, counts = np.unique(y, return_counts=True)
        p = counts / m
        return 1.0 - np.sum(p ** 2)

    def _best_split(self, X, y):
        m, n = X.shape
        if m < self.min_samples_split:
            return None, None

        best_gini = None
        best_feature = None
        best_threshold = None

        current_gini = self._gini(y)

        for feature_idx in range(n):
            x_col = X[:, feature_idx]
            thresholds = np.unique(x_col)
            if thresholds.shape[0] == 1:
                continue

            for t in thresholds:
                left_mask = x_col <= t
                right_mask = x_col > t

                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)

                if n_left < self.min_samples_leaf or n_right < self.min_samples_leaf:
                    continue

                g_left = self._gini(y[left_mask])
                g_right = self._gini(y[right_mask])

                g_split = (n_left * g_left + n_right * g_right) / m

                if best_gini is None or g_split < best_gini:
                    best_gini = g_split
                    best_feature = feature_idx
                    best_threshold = t

        if best_gini is None or best_gini >= current_gini:
            return None, None

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth, rng):
        node = {}
        num_samples = y.shape[0]
        num_labels = np.unique(y).shape[0]

        if num_labels == 1:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        if self.max_depth is not None and depth >= self.max_depth:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        if num_samples < self.min_samples_split:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        feature_idx, threshold = self._best_split(X, y)

        if feature_idx is None:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        left_mask = X[:, feature_idx] <= threshold
        right_mask = X[:, feature_idx] > threshold

        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[right_mask], y[right_mask]

        node["type"] = "internal"
        node["feature_idx"] = feature_idx
        node["threshold"] = threshold
        node["left"] = self._build_tree(X_left, y_left, depth + 1, rng)
        node["right"] = self._build_tree(X_right, y_right, depth + 1, rng)

        return node

    def _class_counts(self, y):
        counts = np.zeros(self.n_classes_, dtype=float)
        for idx, c in enumerate(self.classes_):
            counts[idx] = np.sum(y == c)
        return counts

    def _predict_one_proba(self, x, node):
        if node["type"] == "leaf":
            counts = node["class_counts"]
            total = np.sum(counts)
            if total == 0:
                return np.ones(self.n_classes_) / self.n_classes_
            return counts / total

        if x[node["feature_idx"]] <= node["threshold"]:
            return self._predict_one_proba(x, node["left"])
        else:
            return self._predict_one_proba(x, node["right"])

    def predict_proba(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        proba = np.zeros((n_samples, self.n_classes_), dtype=float)
        for i in range(n_samples):
            proba[i] = self._predict_one_proba(X[i], self.tree_)
        return proba

    def predict(self, X):
        proba = self.predict_proba(X)
        idx = np.argmax(proba, axis=1)
        return self.classes_[idx]


class MyRandomForestClassifier(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        bootstrap=True,
        criterion="gini",
        random_state=None
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.criterion = criterion
        self.random_state = random_state

    def _get_n_features_subspace(self, n_features):
        if isinstance(self.max_features, int):
            return max(1, min(self.max_features, n_features))
        if self.max_features in ["sqrt", None, "auto"]:
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        return n_features

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        n_samples, n_features = X.shape
        n_feats_sub = self._get_n_features_subspace(n_features)

        rng = np.random.RandomState(self.random_state)

        self.trees_ = []
        self.features_idx_ = []

        for i in range(self.n_estimators):
            if self.bootstrap:
                sample_idx = rng.randint(0, n_samples, size=n_samples)
            else:
                sample_idx = np.arange(n_samples)

            feat_idx = rng.choice(n_features, size=n_feats_sub, replace=False)

            X_boot = X[sample_idx][:, feat_idx]
            y_boot = y[sample_idx]

            tree = MyDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                criterion=self.criterion,
                random_state=None if self.random_state is None else self.random_state + i
            )
            tree.fit(X_boot, y_boot)

            self.trees_.append(tree)
            self.features_idx_.append(feat_idx)

        return self

    def predict_proba(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        n_classes = len(self.classes_)

        proba_sum = np.zeros((n_samples, n_classes), dtype=float)

        for tree, feat_idx in zip(self.trees_, self.features_idx_):
            X_sub = X[:, feat_idx]
            proba_sum += tree.predict_proba(X_sub)

        proba_avg = proba_sum / len(self.trees_)
        return proba_avg

    def predict(self, X):
        proba = self.predict_proba(X)
        idx = np.argmax(proba, axis=1)
        return self.classes_[idx]

Бейзлайн

Из отчёта по классам видно, что модель предсказывает только класс 0 (неуспешное аниме). По классу 0 точность и полнота очень высокие, по классу 1 полнота равна 0, F1 тоже 0 — модель вообще не находит «хиты»  
Такая картина говорит о том, что моя реализация случайного леса в текущем виде деградировала до константного классификатора, который выбирает только majority class. При этом ROC-AUC ≈ 0.71 намекает, что какие-то различия между объектами модель всё же ловит (по вероятностям), но при пороге 0.5 она всегда уходит в класс 0  
Подбор гиперпараметров в гипотезе 1 не улучшил ситуацию, потому что при F1=0 для всех комбинаций GridSearchCV просто вернул первый же набор параметров. Для реального применения такую модель использовать нельзя — её нужно дорабатывать: добавлять учёт несбалансированности классов, настраивать порог, либо внимательно проверять логику голосования и предсказаний в реализации леса


In [ ]:
my_rf_base = Pipeline(steps=[
    ("pred", pred),
    ("clf", MyRandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        bootstrap=True,
        criterion="gini",
        random_state=42
    ))
])

my_rf_base.fit(X_train, y_train)

y_pred_my_base = my_rf_base.predict(X_test)
y_proba_my_base = my_rf_base.predict_proba(X_test)[:, 1]

print("Бейзлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_my_base))
print("F1-score:", f1_score(y_test, y_pred_my_base))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my_base))
print(classification_report(y_test, y_pred_my_base))


Бейзлайн
Accuracy: 0.7493775933609959
F1-score: 0.0
ROC-AUC: 0.7149054292901513
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       903
           1       0.00      0.00      0.00       302

    accuracy                           0.75      1205
   macro avg       0.37      0.50      0.43      1205
weighted avg       0.56      0.75      0.64      1205



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Гипотеза 1

Из отчёта видно, что модель по-прежнему предсказывает только класс 0 и полностью игнорирует класс 1 (ни одной верной «популярной» аниме), поэтому F1 по целевому классу равен нулю. Формально гиперпараметры подобраны, но из-за особенностей моей реализации леса и дисбаланса классов подбор параметров не меняет поведение модели: она остаётся по сути константным классификатором

In [ ]:
param_grid_my_rf_1 = {
    "clf__n_estimators": [100, 300],
    "clf__max_depth": [None, 10, 20],
    "clf__min_samples_split": [2, 10, 20],
    "clf__min_samples_leaf": [1, 2, 5],
    "clf__max_features": ["sqrt", "log2"],
}

my_rf_clf_1 = Pipeline(steps=[
    ("pred", pred),
    ("clf", MyRandomForestClassifier(
        bootstrap=True,
        criterion="gini",
        random_state=42
    ))
])

grid_my_rf_1 = GridSearchCV(
    estimator=my_rf_clf_1,
    param_grid=param_grid_my_rf_1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my_rf_1.fit(X_train, y_train)

print("Лучшие параметры:", grid_my_rf_1.best_params_)
print("Лучший F1 на CV:", grid_my_rf_1.best_score_)

best_my_rf_1 = grid_my_rf_1.best_estimator_
y_pred_my_rf_1 = best_my_rf_1.predict(X_test)
y_proba_my_rf_1 = best_my_rf_1.predict_proba(X_test)[:, 1]

print("\nГипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_my_rf_1))
print("F1-score:", f1_score(y_test, y_pred_my_rf_1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my_rf_1))
print(classification_report(y_test, y_pred_my_rf_1))


Лучшие параметры: {'clf__max_depth': None, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}
Лучший F1 на CV: 0.0

Гипотеза 1
Accuracy: 0.7493775933609959
F1-score: 0.0
ROC-AUC: 0.7149054292901513
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       903
           1       0.00      0.00      0.00       302

    accuracy                           0.75      1205
   macro avg       0.37      0.50      0.43      1205
weighted avg       0.56      0.75      0.64      1205



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Гипотеза 2

Однако GridSearchCV снова находит лучший F1 = 0.0, а итоговые метрики на тесте полностью совпадают с бейзлайном: accuracy ≈ 0.75, F1 = 0.0, ROC-AUC ≈ 0.71. Модель по-прежнему всегда предсказывает класс 0 и вообще не находит успешные тайтлы (класс 1), из-за чего F1 по целевому классу остаётся нулевым. То есть логарифмирование признака в рамках моей реализации случайного леса ситуацию не исправило

In [7]:
param_grid_my_rf_1 = {
    "clf__n_estimators": [100, 300],
    "clf__max_depth": [None, 10, 20],
    "clf__min_samples_split": [2, 10, 20],
    "clf__min_samples_leaf": [1, 2, 5],
    "clf__max_features": ["sqrt", "log2"],
}

my_rf_clf_1 = Pipeline(steps=[
    ("pred", pred),
    ("clf", MyRandomForestClassifier(
        bootstrap=True,
        criterion="gini",
        random_state=42
    ))
])
df2 = df.copy()
df2["log_Episodes"] = np.log1p(df2["Episodes"])

num2 = ["log_Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature2 = num2 + categ

X2 = df2[feature2]
y2 = df2["hit"]

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2,
    test_size=0.2,
    random_state=42,
    stratify=y2
)

pred2 = ColumnTransformer(
    transformers=[
        ("num2", StandardScaler(), num2),
        ("categ", OneHotEncoder(handle_unknown="ignore"), categ),
    ]
)

my_rf_clf_2 = Pipeline(steps=[
    ("pred", pred2),
    ("clf", MyRandomForestClassifier(
        bootstrap=True,
        criterion="gini",
        random_state=42
    ))
])

grid_my_rf_2 = GridSearchCV(
    estimator=my_rf_clf_2,
    param_grid=param_grid_my_rf_1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my_rf_2.fit(X2_train, y2_train)

print("Лучшие параметры:", grid_my_rf_2.best_params_)
print("Лучший F1 на CV:", grid_my_rf_2.best_score_)

best_my_rf_2 = grid_my_rf_2.best_estimator_
y_pred_my_rf_2 = best_my_rf_2.predict(X2_test)
y_proba_my_rf_2 = best_my_rf_2.predict_proba(X2_test)[:, 1]

print("\nMyRandomForestClassifier + log(Episodes)")
print("Accuracy:", accuracy_score(y2_test, y_pred_my_rf_2))
print("F1-score:", f1_score(y2_test, y_pred_my_rf_2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_my_rf_2))
print(classification_report(y2_test, y_pred_my_rf_2))


Лучшие параметры: {'clf__max_depth': None, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}
Лучший F1 на CV: 0.0

MyRandomForestClassifier + log(Episodes)
Accuracy: 0.7493775933609959
F1-score: 0.0
ROC-AUC: 0.7149054292901513
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       903
           1       0.00      0.00      0.00       302

    accuracy                           0.75      1205
   macro avg       0.37      0.50      0.43      1205
weighted avg       0.56      0.75      0.64      1205



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Гипотеза 3

Результат снова тот же: лучший F1 на кросс-валидации равен 0.0, accuracy на тесте ≈ 0.75, F1 = 0.0, ROC-AUC ≈ 0.71. Модель по сути остаётся константным классификатором, который всегда выбирает класс 0, даже при более информативном наборе признаков. Это показывает, что проблема не в признаках, а именно в упрощённой реализации моего случайного леса, которая не справляется с дисбалансом классов и практически игнорирует редкий класс успешных аниме.

In [8]:
top_g20 = [g for g, _ in genre_count.most_common(20)]
for g in top_g20:
    col_name = f"genre_{g}"
    if col_name not in df.columns:
        df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))
g_cols20 = [f"genre_{g}" for g in top_g20]

top_s20 = [s for s, _ in studio_count.most_common(20)]
for s in top_s20:
    col_name = f"studio_{s}"
    if col_name not in df.columns:
        df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))
studio_cols20 = [f"studio_{s}" for s in top_s20]

num3 = ["Episodes"] + g_cols20 + studio_cols20
categ3 = ["Source", "Type", "Released_Season", "Demographic"]
feature3 = num3 + categ3

X3 = df[feature3]
y3 = df["hit"]

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3,
    test_size=0.2,
    random_state=42,
    stratify=y3
)

pred3 = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num3),
        ("categ", OneHotEncoder(handle_unknown="ignore"), categ3),
    ]
)

my_rf_clf_3 = Pipeline(steps=[
    ("pred", pred3),
    ("clf", MyRandomForestClassifier(
        bootstrap=True,
        criterion="gini",
        random_state=42
    ))
])

grid_my_rf_3 = GridSearchCV(
    estimator=my_rf_clf_3,
    param_grid=param_grid_my_rf_1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my_rf_3.fit(X3_train, y3_train)

print("Лучшие параметры:", grid_my_rf_3.best_params_)
print("Лучший F1 на CV:", grid_my_rf_3.best_score_)

best_my_rf_3 = grid_my_rf_3.best_estimator_
y_pred_my_rf_3 = best_my_rf_3.predict(X3_test)
y_proba_my_rf_3 = best_my_rf_3.predict_proba(X3_test)[:, 1]

print("\nMyRandomForestClassifier + top-20 жанров/студий")
print("Accuracy:", accuracy_score(y3_test, y_pred_my_rf_3))
print("F1-score:", f1_score(y3_test, y_pred_my_rf_3))
print("ROC-AUC:", roc_auc_score(y3_test, y_proba_my_rf_3))
print(classification_report(y3_test, y_pred_my_rf_3))


Лучшие параметры: {'clf__max_depth': None, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}
Лучший F1 на CV: 0.0

MyRandomForestClassifier + top-20 жанров/студий
Accuracy: 0.7493775933609959
F1-score: 0.0
ROC-AUC: 0.7118196886023775
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       903
           1       0.00      0.00      0.00       302

    accuracy                           0.75      1205
   macro avg       0.37      0.50      0.43      1205
weighted avg       0.56      0.75      0.64      1205



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Библиотечный случайный лес показал, что при корректной настройке гиперпараметров и обработке дисбаланса классов можно существенно повысить качество модели. Бейзлайн давал F1 около 0.35, но после тюнинга и логарифмирования признаков F1 вырос до 0.5 при небольшом падении accuracy. Наилучшие результаты показала гипотеза с логарифмом количества эпизодов — модель стала увереннее находить «хиты», сохраняя общую устойчивость предсказаний   
Собственная реализация случайного леса оказалась неудачной: во всех экспериментах accuracy ≈ 0.75, но F1 = 0. Модель предсказывает только класс 0 и полностью игнорирует редкий класс 1. Это говорит о том, что реализация не обрабатывает дисбаланс и недостаточно точно вычисляет разбиения. Следовательно, библиотечная версия значительно эффективнее и стабильнее для задачи классификации аниме